# Hate Speech and Offensive Language Detection using BERT

The aim of this project is to detect whether a text is considered a hate speech, an offensive language, or neither offensive nor non-offensive using BERT model

**Warning:** It's important to note that dataset contains text that be considered offensive, sexist, discriminative, and racist. Dataset usage is purely for the research purpose.

## Install Requirements

In [1]:
!pip install --upgrade transformers
!pip install --upgrade torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 16.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, DistilBertTokenizer,BertForSequenceClassification, DistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
import pandas as pd
import torch.nn as nn

## Dataset Preparation and Preprocessing

Dataset is from https://www.kaggle.com/datasets/mrmorj/hate-speech-and-offensive-language-dataset using Twitter data. Text is classified as

0: hate-speech
1: offensive-language
2: neither

In [6]:
df = pd.read_csv('labeled_data.csv')

#Merge tweet that has more than 1 row
df['tweet'] = df['tweet'].replace('\n', ' ', regex=True)
# Data cleaning: Remove rows with missing values in any column
df = df.dropna(subset=['count','hate_speech','offensive_language','neither','tweet', 'class'])

df.to_csv('preprocessed_data.csv', index=False)

# Display the first few rows of the dataframe
print(df.head())
print(df.index)


   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  
RangeIndex(start=0, stop=24783, step=1)


Columns in this dataset include:
1. count: number of users who quoted the tweet
2. hate_speech: number of users who judged the tweet to be hate speech
3. offensive_language: number if uses who judged the tweet to be offensive language
4. neither: number of users who judged the tweet to be neither offensive nor non-offensive
5. class: tweet category, either it is considered hate-speech(0), offensive language(1), or neither offensive nor non-offensive(2)
6. tweet: the users tweet

### Train Val Split and Tokenization

The ratio between train and val is 80:20

In [7]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        text = str(self.dataframe.iloc[index]['tweet'])
        label = int(self.dataframe.iloc[index]['class'])

        if pd.notna(text) and pd.notna(label):
          encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
          )

          return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
          }
        else:
            # Return an empty dictionary for skipped rows
            return {}

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define max sequence length based on your data
max_length = 128

# Create datasets and data loaders
train_dataset = CustomDataset(train_df, tokenizer, max_length)
val_dataset = CustomDataset(val_df, tokenizer, max_length)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Load pretrained model

In [8]:
# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Send model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## Model Training and Evaluation

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Set up optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 3

for epoch in range(num_epochs):
    model.train()

    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct_preds = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += criterion(outputs.logits, labels).item()

            preds = torch.argmax(outputs.logits, dim=1)
            correct_preds += torch.sum(preds == labels).item()

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader)
    val_accuracy = correct_preds / len(val_loader.dataset)

    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print(f'Epoch {epoch + 1}/{num_epochs}, '
          f'Loss: {loss:.4f}, '
          f'Validation Loss: {val_loss:.4f}, '
          f'Validation Accuracy: {val_accuracy:.4f}, '
          f'Precision: {precision:.4f}, '
          f'Recall: {recall:.4f}, '
          f'F1 Score: {f1:.4f}')


Epoch 1/3, Loss: 0.5065, Validation Loss: 0.2427, Validation Accuracy: 0.9118, Precision: 0.8951, Recall: 0.9118, F1 Score: 0.8901
Epoch 2/3, Loss: 0.2360, Validation Loss: 0.2577, Validation Accuracy: 0.9084, Precision: 0.8975, Recall: 0.9084, F1 Score: 0.9000
Epoch 3/3, Loss: 0.5384, Validation Loss: 0.2648, Validation Accuracy: 0.9114, Precision: 0.9062, Recall: 0.9114, F1 Score: 0.9084


# Model Evaluation only

In [ ]:
    # Validation loop
    model.eval()
    val_loss = 0.0
    correct_preds = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += criterion(outputs.logits, labels).item()

            preds = torch.argmax(outputs.logits, dim=1)
            correct_preds += torch.sum(preds == labels).item()

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader)
    val_accuracy = correct_preds / len(val_loader.dataset)

    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print(f'Epoch {epoch + 1}/{num_epochs}, '
          f'Loss: {loss:.4f}, '
          f'Validation Loss: {val_loss:.4f}, '
          f'Validation Accuracy: {val_accuracy:.4f}, '
          f'Precision: {precision:.4f}, '
          f'Recall: {recall:.4f}, '
          f'F1 Score: {f1:.4f}')


# Save Model

## To Google Colab

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
%cd /content/gdrive/MyDrive/CCCCC_bert

Mounted at /content/gdrive
/content/gdrive/MyDrive/CCCCC_bert


In [ ]:
torch.save(model.state_dict(), 'model3.pth')

In [ ]:
%cd ../../..

/content


## To Files in Google Colab

In [ ]:
torch.save(model.state_dict(), 'model1.pth')

# Load Model

## From Google Colab

In [9]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [14]:
%cp -av /content/gdrive/MyDrive/CCCCC_bert/model1.pth /content

'/content/gdrive/MyDrive/CCCCC_bert/model1.pth' -> '/content/model1.pth'


In [15]:
model.load_state_dict(torch.load('model1.pth', map_location=device))
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

# Model Demo

In [ ]:
cats = ""
model.eval()

# Example input text
input_text = "i wonder what happen to those guys"  #this is only an example of the input text to test the model

# Tokenize and encode the input text
inputs = tokenizer(input_text, return_tensors='pt')

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

# Get the predicted class probabilities
probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

# Get the predicted class label
predicted_class = torch.argmax(probs, dim=-1).item()

if predicted_class == 0:
  cats = "hate speech"
elif predicted_class ==1:
  cats = "offensive language"
elif predicted_class == 2:
  cats = "neither offensive nor non offensive"

print(f"Predicted Class: {predicted_class}, Category: {cats}, Probabilities: {probs.numpy()}")

Predicted Class: 2, Category: neither offensive nor non offensive, Probabilities: [[0.00839059 0.01282523 0.97878414]]


# Model Demo in Streamlit

In [18]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.9 MB/s eta 0:00:00


In [49]:
%%writefile app.py

import streamlit as st
import torch
from transformers import BertTokenizer, BertForSequenceClassification

model_path = 'model1.pth'
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()


def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.nn.functional.softmax(logits, dim=1)
    predicted_class = torch.argmax(probs, dim=-1).item()
    if predicted_class == 0:
      cats = "Hate Speech 	:rage:"
    elif predicted_class ==1:
      cats = "Offensive Language 	:serious_face_with_symbols_covering_mouth:"
    elif predicted_class == 2:
      cats = "Neither Offensive nor Non Offensive  :neutral_face:"
    return predicted_class, cats, probs.detach().numpy()[0]

# Streamlit app
st.title("BERT Hate Speech and Offensive Language Classification Demo")

# Input text box for user input
user_input = st.text_area("Enter a sentence:")

# Make predictions when the user submits the input
if st.button("Predict"):
    if user_input:
        st.write("Input Sentence:", user_input)
        cat, indicator, probabilities = predict_sentiment(user_input)
        st.write("Category:", cat)
        st.write("Indicator:", indicator)
        st.write("Probabilities:", probabilities)
    else:
        st.warning("Please enter a sentence for prediction.")

Overwriting app.py


In [20]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 3.257s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [50]:
!streamlit run app.py --server.address=localhost &>/content/logs.txt &

In [51]:
!npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.221.245.83
npx: installed 22 in 1.988s
your url is: https://tame-actors-train.loca.lt
